In [ ]:
import time
import os
import copy
import argparse
import pdb
import collections
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torchvision

import model
from anchors import Anchors
import losses
from dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, UnNormalizer, Normalizer
from torch.utils.data import Dataset, DataLoader

import coco_eval
import csv_eval


## Notebook Configurations

In [ ]:
notebook_config = {}
notebook_config['dataset']='coco'                #'Dataset type, must be one of csv or coco.'
notebook_config['coco_path']='../coco/'                #'Path to COCO directory'
notebook_config['csv_train']=None                #'Path to file containing training annotations (see readme)'
notebook_config['csv_classes']=None                #'Path to file containing class list (see readme)'
notebook_config['csv_val']=None                #'Path to file containing validation annotations (optional, see readme)'
notebook_config['depth']=18                #'Resnet depth, must be one of 18, 34, 50, 101, 152'
notebook_config['epochs']=100                #'Number of epochs'
notebook_config['use_gpu'] = True




## Create DataLoaders

In [ ]:
if notebook_config['dataset'] == 'coco':

    if notebook_config['coco_path'] is None:
        raise ValueError('Must provide --coco_path when training on COCO,')

    dataset_train = CocoDataset(notebook_config['coco_path'], set_name='train2017', transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
    dataset_val = CocoDataset(notebook_config['coco_path'], set_name='val2017', transform=transforms.Compose([Normalizer(), Resizer()]))

elif notebook_config['dataset'] == 'csv':

    if notebook_config['csv_train'] is None:
        raise ValueError('Must provide --csv_train when training on COCO,')

    if notebook_config['csv_classes'] is None:
        raise ValueError('Must provide --csv_classes when training on COCO,')


    dataset_train = CSVDataset(train_file=notebook_config['csv_train'], class_list=notebook_config['csv_classes'], transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))

    if notebook_config['csv_val'] is None:
        dataset_val = None
        print('No validation annotations provided.')
    else:
        dataset_val = CSVDataset(train_file=notebook_config['csv_val'], class_list=notebook_config['csv_classes'], transform=transforms.Compose([Normalizer(), Resizer()]))

else:
    raise ValueError('Dataset type not understood (must be csv or coco), exiting.')

sampler = AspectRatioBasedSampler(dataset_train, batch_size=16, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=8, collate_fn=collater, batch_sampler=sampler)

if dataset_val is not None:
    sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
    dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=collater, batch_sampler=sampler_val)


## Create Model

In [ ]:
if notebook_config['depth'] == 18:
    retinanet = model.resnet18(num_classes=dataset_train.num_classes(), pretrained=True)
elif notebook_config['depth'] == 34:
    retinanet = model.resnet34(num_classes=dataset_train.num_classes(), pretrained=True)
elif notebook_config['depth'] == 50:
    retinanet = model.resnet50(num_classes=dataset_train.num_classes(), pretrained=True)
elif notebook_config['depth'] == 101:
    retinanet = model.resnet101(num_classes=dataset_train.num_classes(), pretrained=True)
elif notebook_config['depth'] == 152:
    retinanet = model.resnet152(num_classes=dataset_train.num_classes(), pretrained=True)
else:
    raise ValueError('Unsupported model depth, must be one of 18, 34, 50, 101, 152')        



if notebook_config['use_gpu']:
    retinanet = retinanet.cuda()
    retinanet = torch.nn.DataParallel(retinanet).cuda()

retinanet.training = True


## Prepare for Training

In [ ]:
optimizer = optim.Adam(retinanet.parameters(), lr=1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

loss_hist = collections.deque(maxlen=500)

retinanet.train()
retinanet.module.freeze_bn()
print('Num training images: {}'.format(len(dataset_train)))

## Train

In [ ]:
for epoch_num in range(notebook_config['epochs']):

    retinanet.train()
    retinanet.module.freeze_bn()

    epoch_loss = []

    for iter_num, data in enumerate(dataloader_train):
        try:
            optimizer.zero_grad()

            classification_loss, regression_loss = retinanet([data['img'].cuda().float(), data['annot']])

            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()

            loss = classification_loss + regression_loss

            if bool(loss == 0):
                continue

            loss.backward()

            torch.nn.utils.clip_grad_norm_(retinanet.parameters(), 0.1)

            optimizer.step()

            loss_hist.append(float(loss))

            epoch_loss.append(float(loss))

            print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))

            del classification_loss
            del regression_loss
        except Exception as e:
            print(e)
            continue

    if notebook_config['dataset'] == 'coco':

        print('Evaluating dataset')

        coco_eval.evaluate_coco(dataset_val, retinanet)

    elif notebook_config['dataset'] == 'csv' and notebook_config['csv_val'] is not None:

        print('Evaluating dataset')

        mAP = csv_eval.evaluate(dataset_val, retinanet)


    scheduler.step(np.mean(epoch_loss))    

    torch.save(retinanet.module, '{}_retinanet_{}.pt'.format(notebook_config['dataset'], epoch_num))

retinanet.eval()

torch.save(retinanet, 'model_final.pt'.format(epoch_num))
